# **Brady Analysis**

The point of this exercise to to analyze past nfl data and see if brady elevated his players or not. this is done by first creating a baseline of the player with brady, and how he did without.

In [308]:
#import libraries
import numpy as np
import pandas as pd
import re

In [309]:
#import data undo dataframes for each respective year
#data is from NFLsavant.com
df13 = pd.read_csv('pbp-2013.csv')
df14 = pd.read_csv('pbp-2014.csv')
df15 = pd.read_csv('pbp-2015.csv')
df16 = pd.read_csv('pbp-2016.csv')
df17 = pd.read_csv('pbp-2017.csv')
df18 = pd.read_csv('pbp-2018.csv')
df19 = pd.read_csv('pbp-2019.csv')
df20 = pd.read_csv('pbp-2020.csv')

In [310]:
#create a function to find all players associated with the QB per year
def name_bin(df, name):
    df_bin = []
    for x in df[(df['PlayType'] == 'PASS') & (df['IsIncomplete'] == 0)]['Description']:
        if name in x:
            x_re = re.compile(r'(\w\.\w+)')
            player = x_re.findall(x)
            #print(player)
            if player[1] not in df_bin:
                df_bin.append(player[1])
    return df_bin

#create bins of players for each respective year
bin_2013 = name_bin(df13, QB)
bin_2014 = name_bin(df14, QB)
bin_2015 = name_bin(df15, QB)
bin_2016 = name_bin(df16, QB)
bin_2017 = name_bin(df17, QB)
bin_2018 = name_bin(df18, QB)
bin_2019 = name_bin(df19, QB)
bin_2020 = name_bin(df20, QB)


In [311]:
#create a function to total yards and touchdown for the given year
def get_stats(name, df, year):
    year = str(year)
    year = year[2:]
    #print(year)
    total_yards = 0
    total_tds = 0
    for index, x in df[(df['PlayType'] == 'PASS')  & \
                       (df['GameDate'].str.contains(str(year))) & (df['IsInterception'] == 0) \
                      & (df['IsIncomplete'] == 0) & (df['IsPenaltyAccepted'] == 0)].iterrows():
                           
        if name in x['Description']:
            x_re = re.compile(r'(\w\.\w+)')
            player = x_re.findall(x['Description'])
            if len(player) > 1:
                if (name == player[1]):
                    total_yards += x['Yards']
                    total_tds += x['IsTouchdown']
    return (year, total_yards, total_tds)

#tested for d.amendola  
da_yr, da_yds, da_tds = get_stats('80-D.AMENDOLA', df13, 2013)
print(da_yr, da_yds, da_tds)
            
            

13 0 0


In [312]:
da_test = df19[(df13['PlayType'] == 'PASS') & (df19['Description'].str.contains('A.BROWN'))]
da_test.to_csv('datest.csv')

<ipython-input-312-59d5565a8341>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  da_test = df19[(df13['PlayType'] == 'PASS') & (df19['Description'].str.contains('A.BROWN'))]


In [313]:
#created empty bins for comparison later
pre_brady = {}
with_brady = {}
post_brady = {}

#created lists for easier loop access
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
bins = [bin_2013, bin_2014, bin_2015, bin_2016, bin_2017, bin_2018, bin_2019, bin_2020]
dfs = [df13, df14, df15, df16, df17, df18, df19, df20]




In [314]:
#create the initial bin
for x in bin_2013:
    if x not in with_brady.keys():
        with_brady[x] = [get_stats(x, df13, 2013)]
#print(list(with_brady.items()))
#print(list(with_brady.keys()))

for s in range(1,3):
    for name in bins[s]:
        if name not in bins[s-1]:
            if name not in pre_brady.keys():
                pre_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                pre_brady[name] += [get_stats(name, dfs[s], years[s])]
        elif name not in bins[s+1]:
            if name not in post_brady.keys():
                post_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                post_brady[name] += [get_stats(name, dfs[s], years[s])]
        else:
            if name not in with_brady.keys():
                with_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                with_brady[name] += [get_stats(name, dfs[s], years[s])]
        

In [315]:
for s in range(1,len(bins)-1):
    for name in bins[s]:
        if name not in bins[s-1]:
            if name not in pre_brady.keys():
                pre_brady[name] = []
            for y in range(0,s):
                pre_brady[name] += [get_stats(name, dfs[y], years[y])]
                   
        if name not in bins[s+1]:
            if name not in post_brady.keys():
                post_brady[name] = []
            for y in range(s+1,len(bins)):
                post_brady[name] += [get_stats(name, dfs[y], years[y])]       
                
        if True:
            if name not in with_brady.keys():
                with_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                with_brady[name] += [get_stats(name, dfs[s], years[s])]
#create the final bin              
for name in bin_2020:
    if name not in bin_2019:
        if name not in pre_brady.keys():
            pre_brady[name] = []
        for y in range(0,len(years)-1):
            pre_brady[name] += [get_stats(name, dfs[y], years[y])]
            
    if name not in with_brady.keys():
        with_brady[name] = [get_stats(x, df20, 2020)]
    else:
        with_brady[name] += [get_stats(x, df20, 2020)]
        

In [316]:
#Get rid of brady in the pre bin to avoid confusion
pre_brady = {key:val for key, val in pre_brady.items() if key != QB}

print('pre')
print(pre_brady.items())
print('Post')
print(post_brady.items())
print('with')
print(list(with_brady.items()))


In [341]:
players = pd.DataFrame(columns = ['Name', 'Bucket', 'Year', 'Yards', 'TD'])


for key, value in pre_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'Pre', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)

for key, value in with_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'With', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)

for key, value in post_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'Post', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)
    

    






players = players.sort_values(by = ['Name', 'Year'])
print(players[players['Name'] == 'D.AMENDOLA'])


           Name Bucket Year Yards TD
261  D.AMENDOLA   With   13   633  2
262  D.AMENDOLA   With   14   193  1
263  D.AMENDOLA   With   15   653  3
264  D.AMENDOLA   With   16   238  4
265  D.AMENDOLA   With   17   637  2
521  D.AMENDOLA   Post   18   565  1
522  D.AMENDOLA   Post   19   662  1
523  D.AMENDOLA   Post   20   564  0


In [356]:
master_list = []

for x in bins:
    for i in x:
        if i not in master_list:
            master_list.append(i)
            
avg_df = pd.DataFrame(columns = ['Name', 'Bucket', 'Yards', 'TD'])

for x in master_list:
    pdf = players[players['Name'] == x]
    #print(pdf.head())
    for s in ['Pre', 'With', 'Post']:    
    #['Pre', 'Width', 'Post']:
        bdf = pdf[pdf['Bucket'] == s]
        print(x)
        print(s)
        print(bdf.head())
        if len(bdf) > 1:
            avg_yds = bdf['Yards'].mean()
            print(avg_yds)
            avg_tds = bdf['TD'].mean()
            avg_df = avg_df.append({'Name':x, 'Bucket':s, 'Yards':avg_yds, 'TD':avg_tds}, ignore_index = True)

            
print('final')
print(avg_df.head())
        
    

D.AMENDOLA
Pre
Empty DataFrame
Columns: [Name, Bucket, Year, Yards, TD]
Index: []
D.AMENDOLA
With
           Name Bucket Year Yards TD
261  D.AMENDOLA   With   13   633  2
262  D.AMENDOLA   With   14   193  1
263  D.AMENDOLA   With   15   653  3
264  D.AMENDOLA   With   16   238  4
265  D.AMENDOLA   With   17   637  2
470.8
D.AMENDOLA
Post
           Name Bucket Year Yards TD
521  D.AMENDOLA   Post   18   565  1
522  D.AMENDOLA   Post   19   662  1
523  D.AMENDOLA   Post   20   564  0
597.0
J.EDELMAN
Pre
         Name Bucket Year Yards TD
78  J.EDELMAN    Pre   13  1045  5
79  J.EDELMAN    Pre   14   922  4
80  J.EDELMAN    Pre   15   651  7
81  J.EDELMAN    Pre   16   919  2
82  J.EDELMAN    Pre   17     0  0
707.4
J.EDELMAN
With
          Name Bucket Year Yards TD
266  J.EDELMAN   With   13  1045  5
267  J.EDELMAN   With   14   922  4
268  J.EDELMAN   With   15   651  7
269  J.EDELMAN   With   16   919  2
270  J.EDELMAN   With   18   823  6
908.1666666666666
J.EDELMAN
Post
          

339  M.FLOYD   With   16   452  4
M.FLOYD
Post
        Name Bucket Year Yards TD
509  M.FLOYD   Post   17    78  0
510  M.FLOYD   Post   18    93  0
511  M.FLOYD   Post   19     0  0
512  M.FLOYD   Post   20     0  0
42.75
M.LENGEL
Pre
        Name Bucket Year Yards TD
37  M.LENGEL    Pre   13     0  0
38  M.LENGEL    Pre   14     0  0
39  M.LENGEL    Pre   15     0  0
0.0
M.LENGEL
With
         Name Bucket Year Yards TD
340  M.LENGEL   With   16    18  1
M.LENGEL
Post
         Name Bucket Year Yards TD
517  M.LENGEL   Post   17     0  0
518  M.LENGEL   Post   18    17  1
519  M.LENGEL   Post   19     0  0
520  M.LENGEL   Post   20     0  0
4.25
R.BURKHEAD
Pre
          Name Bucket Year Yards TD
40  R.BURKHEAD    Pre   13     0  0
41  R.BURKHEAD    Pre   14    49  0
42  R.BURKHEAD    Pre   15    94  1
43  R.BURKHEAD    Pre   16   114  0
64.25
R.BURKHEAD
With
           Name Bucket Year Yards TD
341  R.BURKHEAD   With   17   241  3
342  R.BURKHEAD   With   18   131  1
343  R.BURKHEAD   